Firts, we need to import the relevant libraries:
- **pandas** for data analytics
- **time** for time manipulation functions
- **lxml** for working with HTML/XML text
- **requests** for scaping web pages

In [ ]:
import pandas as pd 
import time as t
from lxml import html 
import requests

Assign an empty data frame to a variable called *reviews_df*

In [ ]:
reviews_df = pd.DataFrame()

- Go to www.yelp.com. 
- Type in *Clemson, SC* in the **Near** box and click Search.
- Copy the content inside the URL and paste it into an empty cell. The pasted text will look like this:

https://www.yelp.com/search?find_desc=&find_loc=Clemson%2C+SC&ns=1

- Each browser has a unique signature, presented by an attribute called *user agent*. Many websites do not like being scraped, and they distinguish between a browser visit and automated programs by *user agent*. 

- Open a browser and go to http://www.whatsmyua.info/?source=post_page---------------------------, then copy and paste content from the detected user-agent string box.

Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.1.2 Safari/605.1.15


With this information, we can setup the variables for a web scape of restautants in Philly

In [ ]:
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.1.2 Safari/605.1.15'

In [ ]:
headers = {'User-Agent': user_agent}

In [ ]:
searchlink = 'https://www.yelp.com/search?find_desc=&find_loc=Clemson%2C+SC&ns=1'

In [ ]:
page = requests.get(searchlink, headers = headers)
parser = html.fromstring(page.content)

Find the links for the restaurants listed from the page

In [ ]:
businesslink=parser.xpath('//a[@class="lemon--a__373c0__IEZFH link__373c0__29943 link-color--blue-dark__373c0__1mhJo link-size--inherit__373c0__2JXk5"]')
links = [l.get('href') for l in businesslink]

In [ ]:
for link in links:
    print (link)

List comprehension (Pythonic way of thinking!)

In [ ]:
true_links = [link for link in links if link.startswith('/biz') and \
              not "hrid" in link]
for link in true_links:
    print (link)

We need to create the URLs for these restaurants

In [ ]:
urls = []

for link in true_links:
    url = 'https://www.yelp.com' + str(link)
    print (url)
    urls.append(url)

Copy/paste one of the links to a browser tab to see where it takes you. 

This is only 10. What about the rest of them? 

In [ ]:
page_nums = '//span[@class="lemon--span__373c0__3997G text__373c0__2pB8f text-color--normal__373c0__K_MKN text-align--left__373c0__2pnx_"]'
pg = parser.xpath(page_nums)
for p in pg:
    print(p.text)

In [ ]:
print(pg[0].text)

In [ ]:
int(pg[0].text.split()[3])

What is the pattern?

https://www.yelp.com/search?find_desc=&find_loc=Clemson%2C+SC&ns=1
https://www.yelp.com/search?find_desc=&find_loc=Clemson%2C%20SC&ns=1&start=10
https://www.yelp.com/search?find_desc=&find_loc=Clemson%2C%20SC&ns=1&start=20

Can we use + instead of %20?

Let's make up an URL and test our patterns:
https://www.yelp.com/search?find_desc=&find_loc=Clemson%2C+SC&ns=1&start=60


Putting everything together!

In [ ]:
import pandas as pd 
import time as t
from lxml import html 
import requests

user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.1.2 Safari/605.1.15'
headers = {'User-Agent': user_agent}
urls = []


page_count = int(pg[0].text.split()[3])
suffix = ""
for i in range(1,(page_count + 1)):
    searchlink = 'https://www.yelp.com/search?find_desc=&find_loc=Clemson%2C+SC&ns=1' + suffix
    print(searchlink)
    page = requests.get(searchlink, headers = headers)
    parser = html.fromstring(page.content)
    businesslink=parser.xpath('//a[@class="lemon--a__373c0__IEZFH link__373c0__29943 link-color--blue-dark__373c0__1mhJo link-size--inherit__373c0__2JXk5"]')
    links = [l.get('href') for l in businesslink]
    true_links = [link for link in links if link.startswith('/biz') and not "hrid" in link]
    for link in true_links:
        url = 'https://www.yelp.com' + str(link)
        urls.append(url)
    t.sleep(2)
    suffix = '&start=' + str(10 * i)

In [ ]:
print(len(urls))

In [ ]:
for item in urls:
    print(item)

## Challenge: 

Using this notebook, identify and print out the Yelp URL to all the restaurants in your hometown. 